In [2]:
import random
import numpy
from src.game import gameUr

ur = gameUr.Ur()
ur.gm.move_first(0, 0, 0)

print(ur)

ValueError: list.remove(x): x not in list

In [ ]:
game_length = 14

current_pos = -1

step_count = 0

step_counts=[]

for i in range(100000):

    while current_pos <= game_length:
        step_count += 1
        # print("move to:" + str(current_pos))
        current_pos+= random.choice([0,0,1,1])+random.choice([0,0,1,1])+random.choice([0,0,1,1])

    # print("steps:" + str(step_count))
    current_pos =-1
    step_counts.append(step_count)
    step_count = 0

step_counts


[11,
 10,
 11,
 10,
 10,
 12,
 12,
 14,
 9,
 8,
 10,
 10,
 11,
 10,
 9,
 13,
 12,
 12,
 11,
 11,
 12,
 9,
 12,
 9,
 9,
 12,
 10,
 10,
 10,
 10,
 10,
 11,
 14,
 14,
 12,
 9,
 12,
 9,
 11,
 9,
 8,
 13,
 12,
 10,
 11,
 10,
 7,
 13,
 12,
 11,
 10,
 12,
 13,
 10,
 11,
 10,
 17,
 9,
 14,
 9,
 11,
 10,
 14,
 11,
 11,
 12,
 9,
 11,
 9,
 11,
 10,
 12,
 10,
 10,
 10,
 11,
 9,
 10,
 9,
 9,
 8,
 10,
 11,
 11,
 10,
 10,
 12,
 10,
 11,
 9,
 9,
 11,
 9,
 12,
 14,
 12,
 8,
 7,
 12,
 9,
 10,
 11,
 9,
 8,
 9,
 10,
 10,
 13,
 7,
 10,
 12,
 9,
 11,
 11,
 10,
 11,
 12,
 12,
 10,
 12,
 9,
 14,
 9,
 9,
 10,
 16,
 9,
 10,
 11,
 13,
 12,
 11,
 12,
 12,
 8,
 11,
 10,
 7,
 10,
 10,
 9,
 10,
 12,
 14,
 11,
 12,
 10,
 10,
 9,
 12,
 11,
 7,
 8,
 13,
 10,
 8,
 12,
 10,
 10,
 8,
 10,
 12,
 10,
 12,
 7,
 12,
 8,
 9,
 10,
 10,
 10,
 8,
 11,
 9,
 11,
 10,
 10,
 9,
 13,
 9,
 13,
 12,
 10,
 11,
 11,
 11,
 10,
 14,
 15,
 15,
 10,
 12,
 12,
 10,
 12,
 10,
 15,
 9,
 13,
 13,
 10,
 14,
 11,
 13,
 11,
 11,
 12,
 13,
 12,
 13,


In [ ]:
step_counts.sort()
print("max:" + str(step_counts[-1]))
print("min:" + str(step_counts[0]))
numpy.mean(step_counts)

max:23
min:6


11.00066

In [ ]:
game_length = 14

current_pos = -1

step_count = 0



while current_pos <= game_length:
       step_count += 1
       for i in range(current_pos):
              print(" ", end='')
       print("┠", end='')
       move_dist = random.choice([0,0,1,1])+random.choice([0,0,1,1])+random.choice([0,0,1,1])
       for i in range(move_dist-1):
              print("─", end='')
       print("┨ "+ str(move_dist) )

       current_pos+= move_dist

       print("┠╂", end='')
       for i in range(game_length-1):
              print("┬", end='')
       print("╂┨ "+str(current_pos))
       for i in range(current_pos):
              print(" ", end='')
       print("┃")

 


┠┨ 1
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 0
┃
┠┨ 0
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 0
┃
┠┨ 1
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 1
 ┃
 ┠─┨ 2
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 3
   ┃
   ┠─┨ 2
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 5
     ┃
     ┠┨ 1
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 6
      ┃
      ┠──┨ 3
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 9
         ┃
         ┠─┨ 2
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 11
           ┃
           ┠─┨ 2
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 13
             ┃
             ┠┨ 1
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 14
              ┃
              ┠┨ 1
┠╂┬┬┬┬┬┬┬┬┬┬┬┬┬╂┨ 15
               ┃
